Exercise II. Genomic Signatures
Task 1: Obtain the first chromosome from the genomes of the following organisms: (H. sapiens, M. musculus, R. norvegicus, D. melanogaster, C. elegans and S. cerevisiae). The sequences need to be in fasta format or similar. You may use the database of your choice but you are advised to opt for the UCSC Table Browser.
Task 2: Calculate a table of o/e 2-mer frequencies for a given sequence. Use mononucleotide frequencies as background.
Task 3: Calculate the Manhattan distance of the k-mer frequency for all sequence pairs
Task 4: Return a table with the Manhattan distances alongside your code and a brief discussion of your results. Does the Distance table correspond to what you would expect in terms of distances between the species?

To test the following code all you have to do is download some genomes files in fasta format and save them with a relative path "./data/*.fasta"
The output is going to be a table with the 2-mers frequencies for each given genome, and a table with Manhattan distances between the choosen genomes you use as input. 
Note, that you can change the k and use the same code for any k-mers, you want, but maybe it will take some more time.

In [52]:
from Bio import SeqIO
import glob
import os
import pandas as pd
from scipy.spatial.distance import pdist, squareform

#creating a dictionary with all the kmers in a given sequence
def find_kmers(seq,k):
    kmers = {}
    for i in range(len(seq)-k +1):
        subseq = seq[i:i+k]
        if subseq not in kmers.keys():
            kmers[subseq] = 1
        else:
            kmers[subseq] += 1
    return kmers

#calculate the background frequencies, by calculate the  mononucleotide frequencies
def background_freq(seq):
    seq = seq.upper()
    total = len(seq)
    freq_dict = {
        'A': seq.count('A') / total,
        'C': seq.count('C') / total,
        'G': seq.count('G') / total,
        'T': seq.count('T') / total
    }
    return freq_dict

def parse_fasta(filepath):
    file_kmer_data = {}

#reading all the files I want to use 
    for file in glob.glob(filepath):
        file_name = os.path.basename(file)
        all_kmers = {}
        count = 0

#parsing each file
        for record in SeqIO.parse(file, "fasta"):
            sequence = ''.join([base for base in str(record.seq).upper() if base in 'ACGT'])
#calling the first function to find the 2-mers in every sequence
            kmers = find_kmers(sequence,2)
#calculate the observe k-mers freq, by divide with the lenght of the whole seq for normalization
            kmers_freq = {kmer: count / len(sequence) for kmer, count in kmers.items()}
#calling the second function to calculate mononucleotide frequencies for each seq (expected frequencies)        
            freq_seq = background_freq(sequence)

#calculating o/e frequencies for each k-mer (observe freq/(freq of the first base)*(freq of the second base))            
            normalized_kmers_freq = {
                kmer: freq / (freq_seq.get(kmer[0], 1e-6)* freq_seq.get(kmer[1], 1e-6))\
                for kmer, freq in kmers_freq.items()
            }
#suming all the o/e freq we calculate for every k-mer in a final dict for each file, 
# this step is needed because we didn't parse the files manually  
            for kmer, val in normalized_kmers_freq.items():
                all_kmers.setdefault(kmer, []).append(val)
            count += 1
#we calculating the mean of the freq for every k-mer, to estimate a total freq for every file
        file_kmer_data[file_name] = {k: sum(v) / len(v) for k, v in all_kmers.items()}

#transform the final_dict to a DataFrame
        df = pd.DataFrame.from_dict(file_kmer_data, orient='index').fillna(0)
    
    return df

#You can use the correct relative path, for the fasta files in your directory to use the code
df = parse_fasta("./data/*.fasta")

print(df)

#calculating Manhattan distance and save the output in a new DataFrame
distance_matrix = pdist(df.values, metric='cityblock')

distance_df = pd.DataFrame(squareform(distance_matrix), index=df.index, columns=df.index)

print(distance_df)

                                 CA        AG        GT        TA        AA  \
mouse_chr1.fasta           1.234818  1.214728  0.884626  0.750466  1.059793   
c.elegan_chrI.fasta        1.064757  0.896344  0.823121  0.603778  1.303311   
hg39_chr1.fasta            1.194468  1.097827  0.798628  0.707548  1.125449   
d.melanogaster_chr2.fasta  1.126019  0.890284  0.853149  0.752908  1.218975   
rat_chr1.fasta             1.229868  1.186669  0.880994  0.737624  1.073928   
s.cerevisiae_chrI.fasta    1.124746  0.981633  0.930224  0.762388  1.130446   

                                 GA        AC        AT        TT        GG  \
mouse_chr1.fasta           1.040431  0.885620  0.863618  1.062065  1.192180   
c.elegan_chrI.fasta        1.115394  0.822504  0.857755  1.302336  1.060995   
hg39_chr1.fasta            1.034884  0.802226  0.940917  1.099674  1.254045   
d.melanogaster_chr2.fasta  0.912362  0.855301  0.963295  1.219659  1.055138   
rat_chr1.fasta             1.042834  0.877588  0.87

RESULTS:

        mouse_chr1.fasta  c.elegan_chrI.fasta  \
mouse_chr1.fasta                   0.000000             3.122010   
c.elegan_chrI.fasta                3.122010             0.000000   
hg39_chr1.fasta                    0.917868             2.652247   
d.melanogaster_chr2.fasta          2.951735             1.358070   
rat_chr1.fasta                     0.211573             3.063276   
s.cerevisiae_chrI.fasta            1.991499             1.503113   

                           hg39_chr1.fasta  d.melanogaster_chr2.fasta  \
mouse_chr1.fasta                  0.917868                   2.951735   
c.elegan_chrI.fasta               2.652247                   1.358070   
hg39_chr1.fasta                   0.000000                   2.585243   
d.melanogaster_chr2.fasta         2.585243                   0.000000   
rat_chr1.fasta                    0.775584                   2.930091   
s.cerevisiae_chrI.fasta           1.774615                   1.199446   
                          rat_chr1.fasta  s.cerevisiae_chrI.fasta  
mouse_chr1.fasta                 0.211573                 1.991499  
c.elegan_chrI.fasta              3.063276                 1.503113  
hg39_chr1.fasta                  0.775584                 1.774615  
d.melanogaster_chr2.fasta        2.930091                 1.199446  
rat_chr1.fasta                   0.000000                 1.984641  
s.cerevisiae_chrI.fasta          1.984641                 0.000000  


As we can see from the final table, we conclude that organisms known to be closely related in evolutionary terms, such as rats and mice, have a small Manhattan distance (0.21). In contrast, organisms with a greater evolutionary distance, such as mice and C. elegans, show a much larger distance (3.1). From this, we can infer that biologically related organisms tend to have more similar genome sequences — reflected in smaller genetic distances — and also tend to maintain similar 2-mer frequency patterns.
